In [ ]:
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss

In [ ]:
def hyperopt_objective(X_train, y_train, iterations=50, early_stop=5):
    data_xgb = xgb.DMatrix(X_train,y_train)

    def opt_objective(params):
        params_xgb = {"eta":params["eta"]
                    ,"colsample_bytree":params["colsample_bytree"]
                    # ,"colsample_bynode":params["colsample_bynode"]
                    # ,"alpha":params["alpha"]
                    ,"gamma":params["gamma"]
                    ,"lambda":params["lambda"]
                    ,"min_child_weight":params["min_child_weight"]
                    ,"max_depth":int(params["max_depth"])
                    # ,"subsample":params["subsample"]
                    # ,"rate_drop":params["rate_drop"]

                    ,"booster":"gbtree"
                    ,"objective":'binary:logistic'
                    ,"nthread":-1
                    ,"verbosity":0}
        result = xgb.cv(params_xgb
                        ,data_xgb 
                        ,seed=seed
                        ,nfold=5
                        ,metrics=("logloss") 
                        ,num_boost_round=int(params["num_boost_round"]))
        return result.iloc[-1,2]

    params_grid = {'eta': hp.quniform('eta', 0.01, 0.11, 0.02)
                    ,'num_boost_round': hp.quniform('num_boost_round', 100, 1001, 100)
                    ,'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1.0, 0.1)
                    # ,'colsample_bynode': hp.quniform('colsample_bynode', 0.1, 1.0, 0.1)
                    ,'max_depth': hp.quniform('max_depth', 4, 5, 2)
                    ,'min_child_weight': hp.quniform('min_child_weight', 5, 11, 2)
                    # ,'alpha': hp.quniform('alpha', 0, 2, 0.05)
                    ,'gamma': hp.quniform('gamma', 0, 0.5, 0.05)
                    ,'lambda': hp.quniform('lambda', 0, 2, 0.05)
                    # ,'subsample': hp.quniform('subsample', 0.4, 1.0, 0.1)
                    # ,"rate_drop":hp.quniform("rate_drop",0.0, 1.01, 0.05)
                    }


    def param_hyperopt(num_iter, early_stop):
        trials = Trials()
        early_stop_fn = no_progress_loss(early_stop) # stop if no progress after 50 iterations
        params_best = fmin(opt_objective
                        , space = params_grid
                        , algo = tpe.suggest
                        , max_evals = num_iter
                        , verbose=True
                        , trials = trials
                        , early_stop_fn = early_stop_fn
                        )
        
        params_best['num_boost_round'] = int(params_best['num_boost_round'])
        params_best['max_depth'] = int(params_best['max_depth'])

        print("\n","\n","best params: ", params_best,
                "\n")
        return params_best, trials

    def hyperopt_validation(params, dtrain, dval, y_val):
        model = xgb.train(params, data_xgb)

        y_pred = model.predict(dval)
        y_pred = np.where(y_pred>0.5,1,0)

        f1 = f1_score(y_val, y_pred)

        return f1


    params_best, trials = param_hyperopt(num_iter=iterations, early_stop=early_stop) # number of iterations and number of iterations without progress to stop
    train_f1 = hyperopt_validation(params_best, data_xgb, data_xgb, y_train)
    # validation_f1 = hyperopt_validation(params_best, data_xgb, data_xgb_validation, y_val)
    # print(f'Train F1: {train_f1}, Validation F1: {validation_f1}')
    print(f'Train F1: {train_f1}')

    model = xgb.train(params_best, data_xgb, num_boost_round=int(params_best["num_boost_round"]))

    return model

In [ ]:
hyperparameter_search = "Baysien"
if hyperparameter_search == "Baysien":
    params_chosen = {'colsample_bytree': 0.8, 'eta': 0.02, 'gamma': 0.35000000000000003, 'lambda': 0.15000000000000002, 'max_depth': 4, 'min_child_weight': 6.0, 'num_boost_round': 100}
    search=False
    if search:
        model = hyperopt_objective(X_train, y_train, iterations=100, early_stop=30)
    else:
        model = xgb.train(params_chosen
                            ,xgb.DMatrix(X_train,y_train)
                            ,num_boost_round=params_chosen["num_boost_round"])